## imports

In [34]:
import numpy as np
import os
import cv2
import pandas as pd
import torch
import torch.nn.functional as F

In [3]:
train=[]
for i in range(1, 50000 + 1):
    image_path = os.path.join('train', f"{i}.png")
    train.append(image_path)


#print(train[:5])



In [4]:
# Read the CSV file containing labels
labels = pd.read_csv("trainLabels.csv")

# Convert labels to numeric values
labels['numeric_labels'] = pd.factorize(labels['label'])[0]

# Display the first 5 rows
print(labels.head())

main_labels=torch.tensor(labels['numeric_labels'])

print(main_labels[0:5])

   id       label  numeric_labels
0   1        frog               0
1   2       truck               1
2   3       truck               1
3   4        deer               2
4   5  automobile               3
tensor([0, 1, 1, 2, 3])


## convulational layer build

In [78]:
class Conv:
    def __init__(self,num_filters):
        self.num_filters=num_filters
        self.filters = torch.randn(num_filters, 5, 5) /25
        

    def iterate(self,image):
        h,w=image.shape
        
        for i in range(h-4):  #-4 since we take 5X5x8 as conv kernal.
            for j in range(w-4):
                region=image[i:i+5,j:j+5]
                region=torch.tensor(region)
            yield region,i,j

    def forward(self,inp):
        h,w=inp.shape
        out=torch.zeros((h-4,w-4,self.num_filters))

        for region,i,j in self.iterate(inp):
            out[i,j]=torch.sum(region * self.filters,axis=(1,2))
        return out
        

## maxpool build

In [59]:
class Maxpool:
    def iterate(self,image):
        h,w,num_filters=image.shape

        new_h = h//2
        new_w = w//2

        for i in range(new_h):
            for j in range(new_w):
                region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]   #stride=2 
            yield region,i,j

    def forward(self,inp):
        h,w,num_filters = inp.shape
        out=torch.zeros((h//2,w//2,num_filters))

        for region,i,j in self.iterate(inp):
            out[i,j]=torch.amax(region,axis=(0,1))  
        return out

## softmaxing 

In [60]:
class Softmax:
    def __init__(self,input_len,neurons):
        self.weights= torch.randn(input_len,neurons)/input_len
        self.bias=torch.zeros(neurons)
        
    def forward(self,inp):
        inp=inp.flatten()
        
        
        input_len,neurons = self.weights.shape

        outs=torch.matmul(inp,self.weights)+self.bias

        exp=torch.exp(outs)

        return exp/torch.sum(exp,axis=0)

        
        

In [61]:
conv_1=Conv(8)
mp=Maxpool()
sf=Softmax(14*14*8,10)
    

In [62]:
parameters=[conv_1.filters,sf.weights,sf.bias]

for p in parameters:
    p.requires_grad=True

In [83]:
epoch=10
for _ in range(1):
    for i in range(1):
        im=torch.tensor(cv2.imread(train[i], cv2.IMREAD_GRAYSCALE))
        print("im->",im)
        out=conv_1.forward(im)
        print("after conv->",out)
        out=mp.forward(out)
        print("after mp->",out)
        logits=sf.forward(out)
        print("logits->",logits)
        

        loss=F.cross_entropy(logits,main_labels[i])

        for p in parameters:
            p.grad=None
        loss.backward()

        lr=0.01
        for p in parameters:
            p.data-=lr*p.grad
    #print(loss.item())
print(im)

im-> tensor([[ 61,  44,  48,  ..., 137, 130, 128],
        [ 18,   0,  10,  ...,  94,  89,  94],
        [ 23,   8,  31,  ...,  90,  90,  80],
        ...,
        [172, 153, 156,  ..., 133,  35,  38],
        [146, 128, 143,  ..., 152,  69,  59],
        [150, 136, 146,  ..., 188, 123,  98]], dtype=torch.uint8)
after conv-> tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,


C:\Users\Akshay\AppData\Local\Temp\ipykernel_5636\491017553.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  region=torch.tensor(region)
